In [1]:
import librosa
import numpy as np
from scipy.signal import butter, filtfilt, medfilt
from scipy import signal
import pywt
import pandas as pd

def extract_features(audio_file_path):
    # Load heart sound audio file
    audio_data, sample_rate = librosa.load(audio_file_path, sr=None, mono=True)
    # Select the first 5 seconds of the audio
    audio_data_5s = audio_data[:5 * sample_rate]
    # Normalize the heart sound signal
    audio_data_norm = (audio_data_5s - audio_data_5s.min()) / (audio_data_5s.max() - audio_data_5s.min())
    # Define the filter parameters
    fs = sample_rate  # Sample rate
    lowcut = 25  # Low-pass cut-off frequency in Hz
    highcut = 400  # High-pass cut-off frequency in Hz
    order = 2  # Filter order
    # Compute the Nyquist frequency
    nyq = 0.5 * fs
    # Compute the filter coefficients
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandpass')
    # Apply the filter to the audio data
    audio_data_filtered = filtfilt(b, a, audio_data_norm)
    # Apply median filter to remove spikes
    audio_data_filtered_median = medfilt(audio_data_filtered, kernel_size=3)
    # Define window size and overlap
    window_size = 0.1 # in seconds
    overlap = 0.05 # in seconds
    # Calculate number of samples per window and overlap
    samples_per_window = int(window_size * sample_rate)
    samples_per_overlap = int(overlap * sample_rate)
    # Calculate number of windows
    num_windows = int(np.ceil(len(audio_data_filtered_median) / samples_per_overlap))
    # Initialize feature matrix
    feature_matrix = np.zeros((num_windows, 4))
    # Extract features from each window
    for i in range(num_windows):
        # Calculate start and end indices for current window
        start_index = i * samples_per_overlap
        end_index = start_index + samples_per_window
        # Extract filtered data for current window
        window_data = audio_data_filtered_median[start_index:end_index]
        # Remove spikes
        window_data = signal.medfilt(window_data, kernel_size=5)
        # Extract Hilbert envelope
        hilbert_env = np.abs(signal.hilbert(window_data))
        feature_matrix[i, 0] = np.mean(hilbert_env)
        # Extract Homomorphic envelope
        homomorphic_env = np.abs(signal.hilbert(np.log(np.abs(window_data))))
        feature_matrix[i, 1] = np.mean(homomorphic_env)
        # Extract Wavelet envelope
        coeffs = pywt.wavedec(window_data, wavelet='db4', level=4)
        cA4, cD4, cD3, cD2, cD1 = coeffs
        wavelet_env = pywt.waverec([cA4, cD4, cD3, cD2, cD1], wavelet='db4')
        feature_matrix[i, 2] = np.mean(wavelet_env)
        # Extract Power spectral density envelope
        f, Pxx = signal.welch(window_data, fs=sample_rate, nperseg=samples_per_window)
        psd_env = np.sqrt(Pxx)
        feature_matrix[i, 3] = np.mean(psd_env)
    # Create dataframe with features
    feature_names = ['hilbert_env', 'homomorphic_env', 'wavelet_env', 'psd_env']
    df_features = pd.DataFrame(data=feature_matrix, columns=feature_names)
    df_features['start_time'] = np.arange(0, num_windows * samples_per_overlap, samples_per_overlap) / sample_rate
    df_features['end_time'] = df_features['start_time'] + window_size
    df_features = df_features.drop(['start_time', 'end_time'], axis=1)
    lst_columns_1= []
    for i in range(100):
        for column in df_features.columns:
            lst_columns_1.append(f'Decomp{i}_t_{column}')
    new_data_1= pd.DataFrame(df_features.to_numpy().reshape(1,-1), columns=lst_columns_1)
     
     #Wavelet features
    # wavelet transform
    wavelet = pywt.Wavelet('db4')
    coefficients = pywt.wavedec(audio_data, wavelet, level=4)
    # extracting features
    features_M = []
    for c in coefficients:
        duration = len(c) / sample_rate  
        energy = np.sum(np.square(c))
        peak_amplitude = np.max(np.abs(c))
        mean_amplitude = np.mean(np.abs(c)) 
        std_amplitude = np.std(np.abs(c))
        skew_amplitude = np.abs(np.mean(c**3)) / np.power(np.mean(c**2), 3/2)
        kurtosis_amplitude = np.mean(c**4) / np.power(np.mean(c**2), 2)
        stats = {'Duration': duration, 'Energy': energy, 'Peak Amplitude': peak_amplitude,
                 'Mean Amplitude': mean_amplitude, 'Std Amplitude': std_amplitude,
                 'Skew Amplitude': skew_amplitude, 'Kurtosis Amplitude': kurtosis_amplitude}
        features_M.append(stats)
        final_dfs_M = pd.DataFrame(features_M)
    lst_columns_2 =[]
    for i in range(5):
        for column in final_dfs_M.columns:
            lst_columns_2.append(f'Decomp{i}_{column}')
    new_data_2 = pd.DataFrame(final_dfs_M.to_numpy().reshape(1,-1), columns=lst_columns_2)
    
    # Concatenate the two data frames
    merged_features = pd.concat([new_data_1, new_data_2], axis=1)
    
    return merged_features
    

In [2]:
# Load heart sound audio file
extract_features('./raw_data/training-a/a0007.wav')

/tmp/ipykernel_17517/482277626.py:53: RuntimeWarning: divide by zero encountered in log
  homomorphic_env = np.abs(signal.hilbert(np.log(np.abs(window_data))))
/home/mer/.pyenv/versions/3.10.6/envs/Waves/lib/python3.10/site-packages/scipy/signal/_signaltools.py:2375: RuntimeWarning: invalid value encountered in multiply
  x = sp_fft.ifft(Xf * h, axis=axis)
/home/mer/.pyenv/versions/3.10.6/envs/Waves/lib/python3.10/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/home/mer/.pyenv/versions/3.10.6/envs/Waves/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 200 is greater than input length  = 100, using nperseg = 100
  warnings.warn('nperseg = {0:d} is greater than input length '


,Decomp0_t_hilbert_env,Decomp0_t_homomorphic_env,Decomp0_t_wavelet_env,Decomp0_t_psd_env,Decomp1_t_hilbert_env,Decomp1_t_homomorphic_env,Decomp1_t_wavelet_env,Decomp1_t_psd_env,Decomp2_t_hilbert_env,Decomp2_t_homomorphic_env,...,Decomp3_Std Amplitude,Decomp3_Skew Amplitude,Decomp3_Kurtosis Amplitude,Decomp4_Duration,Decomp4_Energy,Decomp4_Peak Amplitude,Decomp4_Mean Amplitude,Decomp4_Std Amplitude,Decomp4_Skew Amplitude,Decomp4_Kurtosis Amplitude
0,0.036627,4.207414,0.00182,0.00023,0.030177,4.343284,0.001382,0.000232,0.070584,3.936424,...,0.004273,0.157706,136.156814,17.8345,0.027486,0.040239,0.000332,0.000813,1.45723,415.86591
